In [92]:
from bs4 import BeautifulSoup
import requests
import time
from urllib.request import urlopen, Request
from selenium import webdriver

options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
options.add_argument('--headless')
driver = webdriver.Chrome("/usr/lib/chromium-browser/chromedriver", chrome_options=options)

# f = open("book_titles.txt", "x")
# f2 = open("book_descriptions.txt", "x")
# f3 = open("book_genres.txt", "x")


C:\Users\nidhi\AppData\Local\Temp\ipykernel_11984\2349466462.py:11: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("/usr/lib/chromium-browser/chromedriver", chrome_options=options)
C:\Users\nidhi\AppData\Local\Temp\ipykernel_11984\2349466462.py:11: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome("/usr/lib/chromium-browser/chromedriver", chrome_options=options)


In [94]:
import urllib.request

headers = {'User-Agent': 'Mozilla/5.0'}

book_titles = []
book_descriptions = []
book_genres = []
book_authors = []

page_link = "https://www.goodreads.com/list/show/4093.Best_Books_of_the_Decade_2010s?page=1"
# driver.get(page_link)
# soup = BeautifulSoup(driver.page_source, 'html.parser')
page = requests.get(page_link, headers=headers)
soup = BeautifulSoup(page.content, 'html.parser')
idx = 0
for i in range(1):
    books = soup.find_all('a', class_='bookTitle')
    
    for i in books:
        link = "https://goodreads.com" + i['href']
        time.sleep(1)
        resp = requests.get(link, headers=headers)

        book_soup = BeautifulSoup(resp.content, 'html.parser')
        # print(book_soup)
        description = book_soup.find(class_="DetailsLayoutRightParagraph__widthConstrained")
        if description == None:
            if book_soup.find(class_="readable stacked") == None:
                print("No description")
                description = "No description"
            else:
                description = book_soup.find(class_="readable stacked").find('span', style="display:none").get_text()
            
        else:
            description = description.text
            
        book_descriptions.append(description)
        
        book_titles.append(book_soup.find("meta", property="og:title").get('content'))
        genre_buttons = []
    
        genre_buttons = book_soup.find_all(class_="actionLinkLite bookPageGenreLink")
        if(len(genre_buttons) == 0):
            genre_buttons = book_soup.find_all(class_='BookPageMetadataSection__genreButton')
    
        single_book_genres = []
        genre_i = 0
        for j in genre_buttons:
            if(genre_i == 5):
                break
            single_book_genres.append(j.text)
            genre_i += 1
        book_genres.append(single_book_genres)
        if idx == 5:
            break
        idx += 1
       
        author_name = book_soup.find(class_="authorName")
        if author_name == None:
            if book_soup.find(class_="ContributorLink__name") == None:
                author_name = "No author"
                break
            author_name = book_soup.find(class_="ContributorLink__name").get_text()
        else:
            author_name = author_name.get_text()
        book_authors.append(author_name)
        

    page_link = 'https://goodreads.com' + soup.find('a', class_='next_page').get('href')
    # driver.get(page_link)
    # soup = BeautifulSoup(driver.page_source, 'html.parser')
    page = requests.get(page_link, headers=headers)
    soup = BeautifulSoup(page.content, 'html.parser')
    


In [95]:
# f = open("book_titles.txt", "w")
# f2 = open("book_descriptions.txt", "w")
# f3 = open("book_genres.txt", "w")

# f.write(book_titles)
# f.close()
# f2.write(book_descriptions)
# f2.close()
# f3.write(book_genres)
# f3.close()
print(book_titles)
print(book_descriptions)
print(book_genres)
print(book_authors)




['The Fault in Our Stars', 'Divergent (Divergent, #1)', 'Mockingjay (The Hunger Games, #3)', 'Gone Girl', 'The Martian', 'All the Light We Cannot See']
["Despite the tumor-shrinking medical miracle that has bought her a few years, Hazel has never been anything but terminal, her final chapter inscribed upon diagnosis. But when a gorgeous plot twist named Augustus Waters suddenly appears at Cancer Kid Support Group, Hazel's story is about to be completely rewritten.Insightful, bold, irreverent, and raw, The Fault in Our Stars is award-winning author John Green's most ambitious and heartbreaking work yet, brilliantly exploring the funny, thrilling, and tragic business of being alive and in love.", "In Beatrice Prior's dystopian Chicago world, society is divided into five factions, each dedicated to the cultivation of a particular virtue—Candor (the honest), Abnegation (the selfless), Dauntless (the brave), Amity (the peaceful), and Erudite (the intelligent). On an appointed day of every y

Data Preprocessing

In [163]:
from nltk.tokenize import word_tokenize
import contractions


In [165]:

import re
import string
def clean_text(text):
    # remove numbers
    text_nonum = re.sub(r'\d+', '', text)
    # remove punctuations and convert characters to lower case
    text_nopunct = ""
    for char in text_nonum:
        if char not in string.punctuation:
            text_nopunct += char.lower()
        else:    
            text_nopunct += " "
    # substitute multiple whitespace with single whitespace
    # Also, removes leading and trailing whitespaces
    text_no_doublespace = re.sub('\s+', ' ', text_nopunct).strip()
    return text_no_doublespace


In [ ]:
tokenized = []
for description in book_descriptions:
    descrption = contractions.fix(description)
    description = clean_text(description)
    tokens = word_tokenize(description)
    tokenized.append(tokens)

tokenized 

In [ ]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

filtered = []

for word_tokens in tokenized:
    filtered_sentence = []
    for w in word_tokens: 
        if w not in stop_words and w != "s":
            filtered_sentence.append(w)
    filtered.append(filtered_sentence)

filtered


In [ ]:

from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

lemma_descriptions = []
for description in filtered:
    lemma = []
    for w in description:
        #this is so jank
        #ideally we would have to do POS tagging and then pass the actual tag to the function
        w = lemmatizer.lemmatize(w,  'v')
        lemma.append(lemmatizer.lemmatize(w,  'n'))
    lemma_descriptions.append(lemma)    

lemma_descriptions


In [173]:
text_to_vectorize = []
for description in lemma_descriptions:
    book_descrip = ""
    for w in description:
        book_descrip += w + " "
    text_to_vectorize.append(book_descrip)
print(text_to_vectorize)


['despite tumor shrink medical miracle buy year hazel never anything terminal final chapter inscribe upon diagnosis gorgeous plot twist name augustus water suddenly appear cancer kid support group hazel story completely rewrite insightful bold irreverent raw fault star award win author john green ambitious heartbreaking work yet brilliantly explore funny thrill tragic business alive love ', 'beatrice prior dystopian chicago world society divide five faction dedicate cultivation particular virtue—candor honest abnegation selfless dauntless brave amity peaceful erudite intelligent appoint day every year sixteen year old must select faction devote rest live beatrice decision stay family really is—she make choice surprise everyone include highly competitive initiation follow beatrice rename tris struggle alongside fellow initiate live choice make together must undergo extreme physical test endurance intense psychological simulation devastate consequence initiation transform tris must deter

In [177]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

In [180]:
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(text_to_vectorize)
feature_names = vectorizer.get_feature_names_out()
dense = vectors.todense()
denselist = dense.tolist()
df = pd.DataFrame(denselist, columns=feature_names)
df

#slayyyyy

,abnegation,accept,acclaim,afraid,ago,alive,alone,alongside,also,alternate,...,win,without,word,work,world,would,year,yet,york,younger
0,0.000000,0.000000,0.000000,0.0000,0.000000,0.112913,0.000000,0.000000,0.000000,0.000000,...,0.112913,0.000000,0.000000,0.112913,0.000000,0.000000,0.112913,0.112913,0.000000,0.000000
1,0.082479,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.082479,0.164957,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.067634,0.000000,0.135267,0.000000,0.000000,0.000000
2,0.000000,0.089774,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.089774,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.1407,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.0000,0.100849,0.082697,0.100849,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.100849,0.000000,0.000000,0.100849,0.000000,0.082697,0.000000,0.000000
5,0.000000,0.000000,0.083534,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.083534,...,0.068499,0.000000,0.000000,0.068499,0.068499,0.000000,0.000000,0.000000,0.083534,0.083534
